In [ ]:
using Random

pwd()

In [2]:
struct Instance
    name::String
    graphe::BitMatrix # Matrice d'adjacence du graphe
    k::Int
end

mutable struct Solution
    nodecolors::Vector{Int}
end

"""Renvoie le nombre de sommets du graphe de l'instance."""
function Base.length(instance::Instance)
    return size(instance.graphe)[1]
end

function Base.length(solution::Solution)
    return length(solution.nodecolors)
end

In [3]:
function recup_numbre_dans_string(str::String,i::Int) #cette fonction recupere le nombre qui commence à l'indice i dans le string str
    j = i
    while length(str) >= j+1 && str[j+1] >= '0' && str[j+1] <= '9' #verifie si on manipule bien un chiffre
        j+=1
    end
    return (parse(Int64,str[i:j]),j)
end

"""Construit une instance de k-coloration à partir du fichier."""
function read_instance(path::String,k::Int)
    io = open(path)
    b = true
    graphe = BitMatrix
    while (b)
        ligne = readline(io)
        if length(ligne) >= 1 && ligne[1] == 'p'
            n = recup_numbre_dans_string(ligne,8)[1] #nombre de sommets dans le graphe
            graphe = falses(n,n)
        end
        if length(ligne) >= 1 && ligne[1] == 'e'
            u,j = recup_numbre_dans_string(ligne,3)
            v = recup_numbre_dans_string(ligne,j+1)[1]
            graphe[u,v] = true
            graphe[v,u] = true
        end
        if length(ligne) < 1
            b = false
        end
    end
    name = split(path,"/")[2]
    return Instance(name,graphe,k)
end

read_instance

In [4]:
# Définit les instances à évaluer
const instance_list = (
    read_instance("graphs/flat300_26_0.col", 26),
    read_instance("graphs/le450_15c.col", 15),
    read_instance("graphs/dsjc125.1.col", 5),
    read_instance("graphs/dsjc125.9.col", 44),
    read_instance("graphs/dsjc250.1.col", 8),
    read_instance("graphs/dsjc250.9.col", 72),
    read_instance("graphs/dsjc250.5.col", 28),
    read_instance("graphs/dsjc1000.5.col", 86),
    read_instance("graphs/dsjc1000.5.col", 85),
    read_instance("graphs/dsjc1000.5.col", 84)
)

(Instance("flat300_26_0.col", Bool[0 1 … 0 0; 1 0 … 1 1; … ; 0 1 … 0 1; 0 1 … 1 0], 26), Instance("le450_15c.col", Bool[0 0 … 0 0; 0 0 … 1 0; … ; 0 1 … 0 0; 0 0 … 0 0], 15), Instance("dsjc125.1.col", Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 5), Instance("dsjc125.9.col", Bool[0 1 … 1 1; 1 0 … 0 1; … ; 1 0 … 0 0; 1 1 … 0 0], 44), Instance("dsjc250.1.col", Bool[0 0 … 0 0; 0 0 … 1 0; … ; 0 1 … 0 1; 0 0 … 1 0], 8), Instance("dsjc250.9.col", Bool[0 1 … 1 1; 1 0 … 0 1; … ; 1 0 … 0 1; 1 1 … 1 0], 72), Instance("dsjc250.5.col", Bool[0 1 … 1 0; 1 0 … 1 0; … ; 1 1 … 0 0; 0 0 … 0 0], 28), Instance("dsjc1000.5.col", Bool[0 0 … 0 0; 0 0 … 1 1; … ; 0 1 … 0 0; 0 1 … 0 0], 86), Instance("dsjc1000.5.col", Bool[0 0 … 0 0; 0 0 … 1 1; … ; 0 1 … 0 0; 0 1 … 0 0], 85), Instance("dsjc1000.5.col", Bool[0 0 … 0 0; 0 0 … 1 1; … ; 0 1 … 0 0; 0 1 … 0 0], 84))

In [5]:
"""Détermine si les deux sommets indiqués ont la même couleur."""
function is_collision(instance::Instance,solution::Solution,i::Int,j::Int)
    return instance.graphe[i,j] && solution.nodecolors[i] == solution.nodecolors[j]
end

is_collision

In [6]:
"""Calcule le nombre de collisions de la coloration."""
function nbr_collision(instance::Instance,solution::Solution)
    compteur = 0 
    for i = 1:length(instance)
        for j = i+1:length(instance)
            compteur += is_collision(instance,solution,i,j)
        end
    end
    return compteur
end

nbr_collision

In [7]:
"""
Calcule le nombre de collisions au sommet position en lui attribuant couleur.
"""
function nb_collision_sommet_couleur(instance::Instance,solution::Solution,position::Int,couleur::Int) 
    n_collision = 0
    for  i = 1:length(solution) # Les couleurs sont entre 1 et k
        n_collision += instance.graphe[position,i] && solution.nodecolors[i] == couleur
    end
    return n_collision
end

nb_collision_sommet_couleur

In [8]:
"""Calcule l'ordre dans lequel traiter les sommets dans l'heuristique gloutonne (degré décroissant)."""
function calcul_de_ordre_des_sommets(instance::Instance) 
    deg = vec(sum(instance.graphe,dims=1))
    return sortperm(deg,rev=true)
end

"""Détermine la meilleure couleur à attribuer au nœud."""
function meilleure_couleur_locale(instance::Instance,solution::Solution,position::Int)
    best_c = 1
    best_nb_collision = length(solution)
    for c = 2:instance.k
        nb_collision = nb_collision_sommet_couleur(instance,solution,position,c)
        if nb_collision < best_nb_collision
            best_c = c
            best_nb_collision = nb_collision
        end
    end
    return best_c
end

meilleure_couleur_locale

# 1) Heuristique gloutonne

L'algorithme glouton choisi:
 - assigne successivement une couleur à chaque nœud
 - les nœuds sont traités dans l'ordre de degré décroissant
 - en choisissant pour chaque nœud la couleur minimisant le nombre de conflits avec ses voisins.

In [9]:
"""Heuristique gloutonne générant une solution par coloration successive des sommets."""
function glouton(instance::Instance)
    solution = Solution(zeros(Int,length(instance))) # la couleur 0 veut dire non colorée
    ordre_des_sommets = calcul_de_ordre_des_sommets(instance)
    for i ∈ ordre_des_sommets
        c = meilleure_couleur_locale(instance,solution,i)
        solution.nodecolors[i] = c
    end
    return solution
end

glouton

In [10]:
"""Génère une solution aléatoire."""
function sol_alea(instance::Instance)
    return Solution(rand(1:instance.k,length(instance)))
end

sol_alea

In [11]:
begin
    instance = instance_list[3]
    aleatoire = sol_alea(instance)

    println("$(instance.name)\tk=$(instance.k)")
    println("SOLUTION ALÉATOIRE:")
    println(aleatoire.nodecolors)
    print("collisions: ")
    println(nbr_collision(instance,aleatoire))
    gloutonne = glouton(instance)
    println("\nSOLUTION GLOUTONNE:")
    println(gloutonne.nodecolors)
    print("collisions: ")
    println(nbr_collision(instance,gloutonne))
end

dsjc125.1.col	k=5
SOLUTION ALÉATOIRE:


[5, 1, 2, 4, 1, 1, 4, 3, 4, 3, 4, 1, 3, 3, 2, 3, 3, 3, 3, 1, 4, 5, 1, 3, 1, 3, 3, 1, 1, 1, 2, 3, 4, 3, 4, 4, 2, 2, 5, 5, 3, 3, 2, 3, 2, 2, 1, 5, 3, 5, 2, 2, 3, 3, 5, 4, 1, 2, 2, 1, 3, 3, 5, 5, 1, 3, 5, 2, 2, 5, 5, 1, 1, 4, 3, 4, 2, 3, 1, 1, 1, 1, 1, 1, 3, 4, 2, 3, 5, 4, 3, 2, 3, 1, 2, 1, 5, 3, 1, 5, 1, 3, 4, 5, 1, 2, 5, 1, 2, 5, 4, 1, 3, 1, 3, 5, 1, 3, 3, 5, 3, 5, 2, 5, 4]
collisions: 154



SOLUTION GLOUTONNE:
[3, 2, 2, 4, 2, 5, 4, 3, 4, 3, 2, 5, 3, 3, 2, 4, 4, 4, 3, 5, 5, 5, 5, 4, 5, 2, 3, 3, 2, 5, 4, 3, 4, 4, 3, 5, 4, 2, 4, 5, 5, 4, 2, 2, 5, 4, 3, 5, 3, 2, 2, 2, 4, 3, 4, 4, 4, 5, 5, 2, 2, 2, 5, 3, 3, 5, 2, 3, 2, 2, 3, 3, 3, 2, 3, 2, 3, 5, 4, 4, 4, 3, 4, 2, 2, 2, 5, 3, 3, 3, 2, 4, 3, 3, 5, 5, 4, 2, 5, 3, 2, 3, 4, 5, 2, 5, 5, 3, 4, 4, 3, 2, 2, 5, 2, 4, 5, 5, 2, 4, 4, 3, 2, 2, 3]
collisions: 47


## Tests numériques de l'heuristique gloutonne

In [12]:
begin
    nsamples = 10
    println("INSTANCE NAME AND k\tMIN CONFL\tMEAN CONFL\tMAX CONFL\tTOTAL TIME\tTIME BEST SOL\tSOLS PER SECOND")
    for instance ∈ instance_list
        conflicts_samples = Vector{Int}(undef,nsamples)
        time_samples = zeros(nsamples)
        for i = 1:nsamples
            solution = glouton(instance)
            start_time = time_ns()
            conflicts = nbr_collision(instance,solution)
            run_time = time_ns()-start_time
            conflicts_samples[i] = conflicts
            time_samples[i] = run_time*1e-9
        end
        println("$(
            instance.name) k=$(instance.k)\t$(
            minimum(conflicts_samples))\t$(
            sum(conflicts_samples)/nsamples)\t$(
            maximum(conflicts_samples))\t$(
            sum(time_samples))\t$(
            time_samples[argmin(conflicts_samples)])\t$(
            nsamples/sum(time_samples)
        )") # solutions per second irrelevant, isn't it? (since there is no local search loop)
    end
end

INSTANCE NAME AND k	MIN CONFL	MEAN CONFL	MAX CONFL	TOTAL TIME	TIME BEST SOL	SOLS PER SECOND


flat300_26_0.col k=26	226	226.0	226	0.001939	0.0001934	5157.297576070139
le450_15c.col k=15	362	362.0	362	0.0020393	0.00020030000000000002	4903.64340705144
dsjc125.1.col k=5	47	47.0	47	7.57e-5	1.34e-5	132100.3963011889
dsjc125.9.col k=44	21	21.0	21	0.0001271	1.59e-5	78678.20613690007
dsjc250.1.col k=8	88	88.0	88	0.00044200000000000006	4.83e-5	22624.434389140268
dsjc250.9.col k=72	61	61.0	61	0.0007214000000000001	0.0001957	13861.935126143608
dsjc250.5.col k=28	85	85.0	85	0.0013054	0.0001326	7660.487206986365


dsjc1000.5.col k=86	378	378.0	378	0.0249007	0.0023918000000000003	401.5951358797142


dsjc1000.5.col k=85	398	398.0	398	0.0248863	0.002395	401.8275115224039


dsjc1000.5.col k=84	412	412.0	412	0.0243926	0.0022285	409.96039782557006
